In [ ]:
! pip install contractions

In [ ]:
! pip install langdetect

In [ ]:
! pip install wordcloud

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
import contractions
import nltk
import string
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords, words
from langdetect import detect
from langdetect import detect, DetectorFactory
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
df = pd.read_csv('WELFake_Dataset.csv', index_col=False)
df.head(10)

In [101]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
df.isnull().sum().plot(kind="barh")
plt.xlabel('NaN Values')
plt.show()

Above plot shows how much of NaN values present in title and text column

In [ ]:
null = df.isnull().any(axis=1)
null_rows = df[null]

print(null_rows)

This shows the total title which have NaN values, which we'll be dropping in next step

In [92]:
df.dropna(subset=['title'], inplace=True)
df.dropna(subset=['text'], inplace=True)

## EDA

In [ ]:
y = df.label
print(f'Ratio of real and fake news:')
y.value_counts(normalize=True).rename({1: 'real', 0: 'fake'})

Percentage of Real and Fake News:

* 51.03% : Real
* 48.96% : Fake

In [ ]:
class_counts = df['label'].value_counts()

plt.figure(figsize=(8, 5))
class_counts.plot(kind='bar', color=['blue', 'green'], alpha=0.7)
plt.title('Class Distribution (Fake vs. Real News)', fontsize=16)
plt.xlabel('Class', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(rotation=0, fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

From the chart, the counts of fake and real news appear visually similar, which denotes that there is no class imbalance.

In [ ]:
DetectorFactory.seed = 0

def detect_language(text):
    try:
        return detect(text)
    except:
        return "Error"

df['language'] = df['text'].apply(detect_language)
language_counts = df['language'].value_counts()

english_count = language_counts.get('en', 0)
non_english_count = sum(language_counts) - english_count

labels = 'English', 'Non-English'
sizes = [english_count, non_english_count]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
ax1.axis('equal')

plt.show()


In this pie chart, it shows the amount of English and Non-English text present in 'text' column.

From the visualization, it is clear that the non-english content is very small compared to English i.e. 1.9% of the tootal conten. So we can keep neglect this as it'll will not affect our model

In [70]:
DetectorFactory.seed = 0

In [ ]:
def detect_language(title):
    try:
        return detect(title)
    except:
        return "Error"

df['language'] = df['title'].apply(detect_language)
language_counts = df['language'].value_counts()

english_count = language_counts.get('en', 0)
non_english_count = sum(language_counts) - english_count

labels = 'English', 'Non-English'
sizes = [english_count, non_english_count]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
ax1.axis('equal')

plt.show()

Similar as text column pie chart, the non-english class isn't outweighing the other.

In [ ]:
# Counting punctuations
df['punctuation_count'] = df['text'].apply(lambda x: sum([1 for char in x if char in string.punctuation]))

# Counting uppercase letters
df['uppercase_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)

# Counting numerical data
df['numerical_count'] = df['text'].apply(lambda x: sum(1 for char in x if char.isdigit()))

# Plot for punctuatios
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='punctuation_count', hue='label', bins=30, kde=True, palette='Set2')
plt.title('Punctuation Count Distribution (Fake vs. Real)', fontsize=16)
plt.xlabel('Number of Punctuation Marks', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Plot for uppercase letters
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='uppercase_ratio', hue='label', bins=30, kde=True, palette='Set2')
plt.title('Uppercase Letter Ratio (Fake vs. Real)', fontsize=16)
plt.xlabel('Uppercase Letter Ratio', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Plot for numerical data
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='numerical_count', hue='label', bins=30, kde=True, palette='Set2')
plt.title('Numerical Data Count (Fake vs. Real)', fontsize=16)
plt.xlabel('Number of Numerical Characters', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
bins = np.linspace(0, 200, 40)

df['title_len'] = df['title'].apply(len) #length of the title
df['text_len'] = df['text'].apply(len) #length of the text column
df['total_len'] = df['title_len'] + df['text_len'] #total

#bins = np.linspace(0, df['total_len'].max(), 40)

plt.hist(df[df["label"] == 1]["total_len"], bins, alpha=0.5, label="Real", color="red")
plt.hist(df[df["label"] == 0]["total_len"], bins, alpha=0.5, label="Fake", color="green")
plt.legend(loc="upper right")
plt.xlabel('Length of Processed Title')
plt.ylabel('Number of Articles')
plt.title('Distribution of Article Title Lengths by Class')
plt.show()

Text length analysis;
Comparing the length of fake and real news articles.
And we can see there is quite notable difference between real and fake news, real news is a bit longer than fake news.

## Visualization of news title

In [ ]:
titles = ' '.join(title for title in df['title'])
wordcloud = WordCloud(
    background_color='white',
    max_words=300,
    width=800,
    height=400,
).generate(titles)

plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

From the above visualization we can infer that most of the news title is related to elections and some of the big leader's names around the world.

In [98]:
# Extracting 2 dataframes from df(dataset) based on the label: 0 & 1 which are fake and real news

fake_news = df[df['label'] == 0]
real_news = df[df['label'] == 1]

In [ ]:
real_news

In [ ]:
fake_news

## Visualization of only Fake News of title column

In [ ]:
fake_texts = ' '.join(text for text in fake_news['title'])
wordcloud = WordCloud(
    background_color='white',
    max_words=300,
    width=800,
    height=400,
).generate(fake_texts)

plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

## Visualization of only Real News of title_text column

In [ ]:
real_texts = ' '.join(text for text in real_news['title'])
wordcloud = WordCloud(
    background_color='white',
    max_words=300,
    width=800,
    height=400,
).generate(real_texts)

plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show

In [ ]:
# Extract n-grams and their frequencies
def get_top_ngrams(corpus, n=2, top_n=20):
    vectorizer = CountVectorizer(ngram_range=(n, n), stop_words='english')
    X = vectorizer.fit_transform(corpus)
    ngrams = vectorizer.get_feature_names() 
    counts = X.toarray().sum(axis=0)
    ngram_freq = dict(zip(ngrams, counts))
    sorted_ngrams = sorted(ngram_freq.items(), key=lambda x: x[1], reverse=True)[:top_n]
    return sorted_ngrams

# Top 20 bigrams and trigrams
fake_bigrams = get_top_ngrams(df[df['label'] == 1]['title'], n=2, top_n=20)
real_bigrams = get_top_ngrams(df[df['label'] == 0]['title'], n=2, top_n=20)

fake_trigrams = get_top_ngrams(df[df['label'] == 1]['title'], n=3, top_n=20)
real_trigrams = get_top_ngrams(df[df['label'] == 0]['title'], n=3, top_n=20)

print("Top 20 Bigrams in Fake News Titles:")
for bigram, freq in fake_bigrams:
    print(f"{bigram}: {freq}")

print("\nTop 20 Bigrams in Real News Titles:")
for bigram, freq in real_bigrams:
    print(f"{bigram}: {freq}")

print("\nTop 20 Trigrams in Fake News Titles:")
for trigram, freq in fake_trigrams:
    print(f"{trigram}: {freq}")

print("\nTop 20 Trigrams in Real News Titles:")
for trigram, freq in real_trigrams:
    print(f"{trigram}: {freq}")


In [ ]:
# Visualize n-grams
def plot_ngrams(ngrams, title):
    labels, values = zip(*ngrams)
    plt.figure(figsize=(10, 6))
    plt.barh(labels, values, color='skyblue')
    plt.title(title, fontsize=16)
    plt.xlabel('Frequency', fontsize=14)
    plt.ylabel('Bigrams', fontsize=14)
    plt.gca().invert_yaxis() 
    plt.show()

plot_ngrams(fake_bigrams, 'Top 20 Bigrams in Fake News Titles')
plot_ngrams(real_bigrams[:20], 'Top 20 Bigrams in Real News Titles')

In [ ]:
fake_trigrams = get_top_ngrams(df[df['label'] == 1]['title'], n=3, top_n=20)
real_trigrams = get_top_ngrams(df[df['label'] == 0]['title'], n=3, top_n=20)

plot_ngrams(fake_trigrams, 'Top 20 Trigrams in Fake News Titles')
plot_ngrams(real_trigrams, 'Top 20 Trigrams in Real News Titles')


In [ ]:
# Sentiment polarity
df['sentiment_polarity'] = df['title'].apply(lambda x: TextBlob(x).sentiment.polarity)

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='sentiment_polarity', hue='label', bins=30, kde=True, palette='Set2')
plt.title('Sentiment Polarity Distribution (Fake vs. Real News)', fontsize=16)
plt.xlabel('Sentiment Polarity', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
df

# Pre-processing

In [29]:
#df.isnull().sum().plot(kind="barh")
#plt.show()

class TextProcessor:
    def __init__(self):
        self.stemmer = PorterStemmer()
        self.lemmatizer = WordNetLemmatizer()
        self.stopWords = set(stopwords.words('english'))
        self.englishWords = set(nltk.corpus.words.words())
        self.url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    def url_count(self, text): #Counts URLs
        return len(re.findall(self.url_pattern, text))

    def remove_urls(self, text): #Remove
        return re.sub(self.url_pattern, '', text)

    #def to_lowercase(self, text):
        #return [token.lower() for token in text]

    def remove_contraction(self, text): #Expands contractions
        return ' '.join([contractions.fix(word) for word in text.split()])

    def clean_tokenize_text(self, text): #Removes non_alphanumeric characters and tokenizes the text into words
        cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        cleaned_text = re.sub(r',+', ' ', cleaned_text)
        return word_tokenize(cleaned_text)

    #def stem(self, tokens):
        #return [self.stemmer.stem(token) for token in tokens]

    # def lemmatize(self, tokens): #Reduces each token to its base form using WordNet
    #     return [self.lemmatizer.lemmatize(token) for token in tokens]
    
    def pos_lemmatize(self, tokens): # POS-aware lemmatization
        pos_tags = nltk.pos_tag(tokens)  # POS tags
        return [self.lemmatizer.lemmatize(token, pos=self.get_wordnet_pos(tag)) for token, tag in pos_tags]

    def get_wordnet_pos(self, pos_tag): # Map NLTK POS tags to WordNet POS tags
        if pos_tag.startswith('J'):
            return wordnet.ADJ
        elif pos_tag.startswith('V'):
            return wordnet.VERB
        elif pos_tag.startswith('N'):
            return wordnet.NOUN
        elif pos_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
        
    def handle_negations(self, tokens): #Handling Negations
        negations = ["not", "no", "never", "n't"]
        for i in range(len(tokens) - 1):
            if tokens[i] in negations:
                tokens[i] = tokens[i] + '_' + tokens[i + 1]
                tokens[i + 1] = ''
        return [token for token in tokens if token]

    def remove_numbers(self, tokens): #Remove Numbers
        return [token for token in tokens if not token.isdigit()]
    
    def remove_special_characters(self, text): #Remove Special Characters
        return re.sub(r'[^\w\s]', '', text)

    def correct_spelling(self, tokens):
        return [str(TextBlob(token).correct()) for token in tokens]

    def remove_stopWords(self, tokens): #Removes common stopwords
        return [w for w in tokens if w not in self.stopWords]


Custom well mannered class which consist of a comprehensive set of tools for text pre-processing, which includes: Removing url, ciontraction, tokenization, lemmatization and removing stopwords.

We used lemmatization instead of stemming because stemming just simply removed the only last 3 letters of the token which in our dataset introduced non-english words, as lemmatization reduces the words back to its root word.

In [32]:
from tqdm import tqdm

text_processor = TextProcessor()

def process_text_column(column):
    tqdm.pandas(desc="Processing Articles")
    def process(text):
        original_text = text 
        #print(f"Original text: {original_text}")
        text = text_processor.remove_urls(text).lower()
        #print(f"After remove_urls: {text}")

        if not isinstance(text, str) or not text.strip():
            print(f"Skipping due to empty or non-string text: {original_text}")
            return []

        try:
            # Skip non-English text
            if detect(text) != 'en':
                print(f"Skipping non-English text: {original_text}")
                return []
        except Exception as e:
            print(f"Error detecting language for text: {original_text}. Error: {e}")
            return []

        text = text_processor.remove_contraction(text)
        #print(f"After remove_contraction: {text}")

        text = text_processor.remove_special_characters(text)

        tokens = text_processor.clean_tokenize_text(text)
        #print(f"After clean_tokenize_text: {tokens}")
       
        tokens = text_processor.remove_numbers(tokens)

        tokens = text_processor.handle_negations(tokens)

        tokens = text_processor.remove_stopWords(tokens)
        #print(f"After remove_stopWords: {tokens}")

        tokens = text_processor.correct_spelling(tokens)

        #tokens = text_processor.stem(tokens)
        #print(f"After stem: {tokens}")

        tokens = text_processor.pos_lemmatize(tokens)

        #tokens = text_processor.lemmatize(tokens)
        print(f"After lemmatize: {tokens}") 
        return tokens

    return column.progress_apply(process)




Defined a function which processes text data column-wise, as it is made to work with our custom 'TextProcessor' class. It consist of some debug code which when executed gives all the modifications our dataset will go through while doing this pre-processing.

In [33]:
#df_sampled = df.sample(frac=0.1, random_state=42)

In [ ]:
df['processed_title'] = process_text_column(df['title'])

In [ ]:
df['processed_text'] = process_text_column(df['text'])

In [ ]:
df['processed_title']

In [ ]:
df['processed_text']

In [ ]:
df.head()

In [42]:
df.to_csv('preprocessed_welfake.csv', index=False)


In [8]:
df = pd.read_csv('FINAL_FINAL_FINALLLLLLLLLLLLLL_preprocessed_welfake.csv', index_col=False)

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Relevant features for the model
feature_columns = [
    'punctuation_count', 'uppercase_ratio', 'numerical_count',
    'sentiment_polarity', 'title_len', 'text_len', 'total_len'
]
X = df[feature_columns]
y = df['label'] 

# DataSplit
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, log_loss

lr = LogisticRegression(random_state=42)
lr.fit(X_train_scaled, y_train)

y_pred_lr = lr.predict(X_test_scaled)
print("Logistic Regression Performance:")
print(classification_report(y_test, y_pred_lr))
print("ROC-AUC:", roc_auc_score(y_test, lr.predict_proba(X_test_scaled)[:, 1]))

lr_acc = accuracy_score(y_test, y_pred_lr)
lr_loss = log_loss(y_test, lr.predict_proba(X_test_scaled))

### Support Vector Machine

In [ ]:
from sklearn.svm import SVC

svm = SVC(probability=True, random_state=42)
svm.fit(X_train_scaled, y_train)

y_pred_svm = svm.predict(X_test_scaled)
print("SVM Performance:")
print(classification_report(y_test, y_pred_svm))
print("ROC-AUC:", roc_auc_score(y_test, svm.predict_proba(X_test_scaled)[:, 1]))

svm_acc = accuracy_score(y_test, y_pred_svm)
svm_loss = log_loss(y_test, svm.predict_proba(X_test_scaled))


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
print("Random Forest Performance:")
print(classification_report(y_test, y_pred_rf))
print("ROC-AUC:", roc_auc_score(y_test, rf.predict_proba(X_test)[:, 1]))

rf_acc = accuracy_score(y_test, y_pred_rf)
rf_loss = log_loss(y_test, rf.predict_proba(X_test))


### XGB-Classifier

In [ ]:
pip install xgboost

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_scaled, y_train)

y_pred_xgb = xgb.predict(X_test_scaled)
print("XGBoost Performance:")
print(classification_report(y_test, y_pred_xgb))
print("ROC-AUC:", roc_auc_score(y_test, xgb.predict_proba(X_test_scaled)[:, 1]))

xgb_acc = accuracy_score(y_test, y_pred_xgb)
xgb_loss = log_loss(y_test, xgb.predict_proba(X_test_scaled))


## TF-IDF

In [78]:
from scipy.sparse import hstack


df['combined_processed_text'] = df['processed_title'] + df['processed_text']

# vectorization
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
tfidf_features = tfidf_vectorizer.fit_transform(df['combined_processed_text'])

# use numerical features only
numerical_features = ['punctuation_count', 'uppercase_ratio', 'numerical_count', 
                      'sentiment_polarity', 'text_len', 'title_len', 'total_len']
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[numerical_features])

# combine TF-IDF and scaled num features
feature_matrix = hstack([tfidf_features, scaled_features])

X = feature_matrix
y = df['label'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


### Random Forest Classifier

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

rf_test_pred = rf.predict(X_test)

rf_test_pred_ensemble = rf.predict_proba(X_test)
rf_train_pred_ensemble = rf.predict_proba(X_train)

print("Accuracy:", accuracy_score(y_test, rf_test_pred))
print("\nClassification Report:\n", classification_report(y_test, rf_test_pred))

rf_tfidf_acc = accuracy_score(y_test, rf_test_pred)
rf_tfidf_loss = log_loss(y_test, rf_test_pred_ensemble)

### XGB-Classifier

In [ ]:
xgb = XGBClassifier(
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=6, 
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
xgb.fit(X_train, y_train)

xgb_test_pred = xgb.predict(X_test)

xgb_test_pred_ensemble = xgb.predict_proba(X_test)
xgb_train_pred_ensemble = xgb.predict_proba(X_train)

print("Accuracy:", accuracy_score(y_test, xgb_test_pred))
print("\nClassification Report:\n", classification_report(y_test, xgb_test_pred))

xgb_tfidf_acc = accuracy_score(y_test, xgb_test_pred)
xgb_tfidf_loss = log_loss(y_test, xgb_test_pred_ensemble)

## LSTM

In [93]:
texts = df['combined_processed_text'].apply(lambda x: ' '.join(x))  # join tokens into sentences
labels = df['label']

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences to ensure uniform length
max_length = 200
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.3, random_state=42)



In [ ]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_shape=(max_length,)),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    verbose=1
)


In [ ]:
lstm_test_probs = model.predict(X_test)
lstm_train_probs = model.predict(X_train)

lstm_test_pred = (lstm_test_probs > 0.5).astype("int32")
lstm_train_pred = (lstm_train_probs > 0.5).astype("int32")

print("Accuracy:", accuracy_score(y_test, lstm_test_pred))
print("\nClassification Report:\n", classification_report(y_test, lstm_test_pred))

lstm_acc = accuracy_score(y_test, lstm_test_pred)
lstm_loss = log_loss(y_test, lstm_test_probs)

print(f"Loss: {lstm_loss:.4f}")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.show()


## Ensemble Model - 1

In [ ]:
train_stack = np.hstack([rf_train_pred_ensemble , xgb_train_pred_ensemble, lstm_train_pred])
test_stack = np.hstack([rf_test_pred_ensemble , xgb_test_pred_ensemble, lstm_test_pred])

meta_model = LogisticRegression()
meta_model.fit(train_stack, y_train)

ensemble_preds = meta_model.predict(test_stack)

print("Ensemble Model Accuracy:", accuracy_score(y_test, ensemble_preds))
print("\nClassification Report:\n", classification_report(y_test, ensemble_preds))

ensemble1_acc = accuracy_score(y_test, ensemble_preds)
ensemble1_loss = log_loss(y_test, meta_model.predict_proba(test_stack))

## Ensemble Model - 2 (Weighted Averaging)

In [ ]:
rf_probs = rf_test_pred_ensemble 
xgb_probs = xgb_test_pred_ensemble
lstm_probs = lstm_test_pred 

# weights
rf_weight = 0.6 
xgb_weight = 0.2
lstm_weight = 0.1

ensemble_probs = (
    rf_weight * rf_probs + xgb_weight * xgb_probs + lstm_weight * lstm_probs
)

ensemble_preds = np.argmax(ensemble_probs, axis=1)

accuracy = accuracy_score(y_test, ensemble_preds)
print("Ensemble Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, ensemble_preds))

ensemble2_acc = accuracy_score(y_test, ensemble_preds)
ensemble2_loss = log_loss(y_test, meta_model.predict_proba(test_stack))

## BERT

In [ ]:
pip install transformers torch


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
df['combined_text_title'] = df['title'] + df['text']
df

In [59]:
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
max_len = 128
texts = df['combined_text_title'].tolist()
labels = df['label'].tolist()

X_train, X_val, y_train, y_val = train_test_split(texts, labels, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_dataset = NewsDataset(X_train, y_train, tokenizer, max_len)
val_dataset = NewsDataset(X_val, y_val, tokenizer, max_len)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Binary classification
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
loss_fn = torch.nn.CrossEntropyLoss()

In [64]:
from tqdm import tqdm

def train_model(model, data_loader, optimizer, loss_fn, device):
    model.train()
    total_loss, total_correct = 0, 0

    progress_bar = tqdm(data_loader, desc="Training", leave=False)
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()
        total_correct += (torch.argmax(logits, dim=1) == labels).sum().item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        progress_bar.set_postfix(loss=loss.item())

    accuracy = total_correct / len(data_loader.dataset)
    return total_loss / len(data_loader), accuracy

In [65]:
def evaluate_model(model, data_loader, loss_fn, device, save_logits=False, logits_file="bert_logits.npy"):
    model.eval()
    total_loss, total_correct = 0, 0
    all_logits = []

    progress_bar = tqdm(data_loader, desc="Evaluating", leave=False)
    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()
            total_correct += (torch.argmax(logits, dim=1) == labels).sum().item()

            if save_logits:
                all_logits.append(logits.cpu().numpy())

            progress_bar.set_postfix(loss=loss.item())

    if save_logits:
        all_logits = np.vstack(all_logits)
        np.save(logits_file, all_logits)
        print(f"Logits saved to '{logits_file}'")

    accuracy = total_correct / len(data_loader.dataset)
    return total_loss / len(data_loader), accuracy

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 10

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    train_loss, train_acc = train_model(model, train_loader, optimizer, loss_fn, device)
    val_loss, val_acc = evaluate_model(model, val_loader, loss_fn, device)

    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}')

In [ ]:
y_preds, y_true = [], []
bert_acc = 0.0
bert_loss = float('inf')
total_loss = 0.0 
total_correct = 0  
model.eval()
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()
        total_correct += (torch.argmax(logits, dim=1) == labels).sum().item()

        y_preds.extend(torch.argmax(logits, dim=1).tolist())
        y_true.extend(labels.tolist())

print("Classification Report:")
print(classification_report(y_true, y_preds))

In [ ]:
val_loss = total_loss / len(val_loader)
val_acc = total_correct / len(val_loader.dataset)

if val_acc > bert_acc:
    bert_acc = val_acc
if val_loss < bert_loss:
    bert_loss = val_loss

bert_acc = val_acc
bert_loss = val_loss

print(f'Best Validation Accuracy: {bert_acc:.4f}')
print(f'Best Validation Loss: {bert_loss:.4f}')

## Ensemble Model - 3

In [ ]:
min_len = min(len(bert_probs), len(lstm_probs), len(y_test))

# Resize
bert_probs = bert_probs[:min_len]
lstm_probs = lstm_probs[:min_len]
y_test = y_test[:min_len]

# Weights
bert_weight = 0.9
lstm_weight = 0.1

ensemble_probs_bert_lstm = bert_weight * bert_probs + lstm_weight * lstm_probs
ensemble_preds_bert_lstm = (ensemble_probs_bert_lstm > 0.5).astype("int32")

ensemble3_acc = accuracy_score(y_test, ensemble_preds_bert_lstm)
ensemble3_loss = log_loss(y_test, ensemble_probs_bert_lstm)

print("BERT + LSTM Ensemble Accuracy:", ensemble3_acc)
print("BERT + LSTM Ensemble Loss:", ensemble3_loss)
print("\nClassification Report:\n", classification_report(y_test, ensemble_preds_bert_lstm))


## Ensemble model - 4

In [ ]:
min_len = min(len(bert_probs), len(lstm_probs), len(rf_probs), len(xgb_probs), len(y_test))

bert_probs = bert_probs[:min_len]
lstm_probs = lstm_probs[:min_len]
rf_probs = rf_probs[:min_len]
xgb_probs = xgb_probs[:min_len]
y_test = y_test[:min_len]

# Weights
bert_weight = 0.7
lstm_weight = 0.1
rf_weight = 0.1
xgb_weight = 0.1

ensemble_probs_all = (
    bert_weight * bert_probs +
    lstm_weight * lstm_probs +
    rf_weight * rf_probs +
    xgb_weight * xgb_probs
)

ensemble_preds_all = (ensemble_probs_all > 0.5).astype("int32")

ensemble4_acc = accuracy_score(y_test, ensemble_preds_all)
ensemble4_loss = log_loss(y_test, ensemble_probs_all)

print("BERT + LSTM + Random Forest + XGBoost Ensemble Accuracy:", ensemble4_acc)
print("BERT + LSTM + Random Forest + XGBoost Ensemble Loss:", ensemble4_loss)
print("\nClassification Report:\n", classification_report(y_test, ensemble_preds_all))

In [ ]:
import matplotlib.pyplot as plt

model_names = ['Logistic Regression', 'SVM', 'Random Forest', 'XGBoost', 
               'RF TF-IDF', 'XGB TF-IDF', 'LSTM', 'Ensemble 1', 'Ensemble 2', 
               'BERT', 'Ensemble 3', 'Ensemble 4']

accuracies = [
    lr_acc, svm_acc, rf_acc, xgb_acc,
    rf_tfidf_acc, xgb_tfidf_acc, lstm_acc, ensemble1_acc, 
    ensemble2_acc, bert_acc, ensemble3_acc, ensemble4_acc
]

losses = [
    lr_loss, svm_loss, rf_loss, xgb_loss,
    rf_tfidf_loss, xgb_tfidf_loss, lstm_loss, ensemble1_loss, 
    ensemble2_loss, bert_loss, ensemble3_loss, ensemble4_loss
]

accuracies_percentage = [acc * 100 for acc in accuracies]

plt.figure(figsize=(12, 6))
plt.barh(model_names, accuracies_percentage, color='skyblue')
plt.xlabel('Accuracy (%)')
plt.title('Model Accuracies (in %)')
plt.xlim(0, 100)
plt.gca().invert_yaxis()
plt.show()

# Plot Loss
plt.figure(figsize=(12, 6))
plt.barh(model_names, losses, color='lightcoral')
plt.xlabel('Log Loss')
plt.title('Model Losses')
plt.gca().invert_yaxis()
plt.show()

In [99]:
df.to_csv('final_preprocessed_welfake_dataset.csv', index=False)
